<h4 class='prehead'>SM286D &middot; Introduction to Applied Mathematics with Python &middot; Spring 2020 &middot; Foraker/Traves/Uhan</h4>

<h3 class='lesson'>Project 5.</h3>

<h1 class='lesson_title'>Social Network Analysis</h1>

__Mathematical goals.__  Graphs, measures of centrality, approximation of eigenvectors using the power method, PageRank algorithm.

__Programming goals.__ Loops, dictionaries, NetworkX, navigating documentation.

---

## Background

Edward Snowden is a computer professional who worked as a contractor for the NSA. In 2013, he provided a massive amount of classified information to journalists who later published articles in *The Guardian* and *The Washington Post*. The information leaked by Snowdon revealed the existence of many mass surveillance programs operated by the NSA and our allies. Charged with violating the Espionage Act of 1917 and the theft of government property, Snowden sought asylum in Russia, where he continues to live today. 

Snowden's actions remain deeply controversial; many see him as a traitor, while others see him as a patriot and a whistleblower. Whatever your perspective, Snowden has "fueled debates over mass surveillance, government secrecy and the balance between national security and information privacy." (Wikipedia page on Edward Snowden, retrieved 08 OCT 2016) A documentary, _Citizenfour_, about Snowden and these issues won the 2015 Academy Award for Best Documentary Feature; Figure 1 is pulled from a screenshot from the film. In 2016 Oliver Stone released another biopic about Snowden.  

<img src="img/Edward_Snowden-2.jpg" alt="Drawing" style="width: 220px;"/>

<center> Figure 1. Edward Snowden (Laura Poitras of Praxis Films) </center>

One program run by the NSA involved getting meta-data on phone calls between individuals in the United States. This data didn't contain details of conversations but did help government agencies build a graph describing the network of relationships among persons of interest. A graph consists of a collection of vertices joined by edges connecting one vertex to another. For instance, each vertex might correspond to one of the 9/11 hijackers or their contacts, and edges between vertices might represent known associations between the terrorists and their accomplices. [Valdis Krebs](http://www.orgnet.com/hijackers.html) built such a graph, pictured in Figure 2. His paper, _Mapping Networks of Terrorist Cells_, describes some of the connections between the terrorists. His goal in the paper is to use the graph to identify the key members of the terrorist network. This kind of Social Network Analysis has been used to study a wide range of networks, including: economic networks, co-authorship networks in academia, networks of lobbyists, online social networks (e.g. Twitter or Facebook), and political networks. See http://www.orgnet.com/cases.html for details of these applications.

<img src="img/hijackers_graph_with_0_based_labels2.jpg" alt="Drawing" style="width: 600px;"/>

<center> Figure 2. A graph $G$ presenting the social network of the 9/11 hijackers (Krebs, http://www.orgnet.com/hijackers.html; the numerical labels are new). </center>

Following Krebs, our goal in this assignment is to use several measures of centrality to identify the key members of the 9/11 terrorist network. First we need to associate a vertex with each conspirator. The numbered list of vertices and their associated conspirators is presented in the dictionary `names` in the code provided below. 

In [ ]:
# Import numpy  
import numpy as np

# Define a dictionary that maps numbers to names
# a * next to a name indicates that the person is suspected of using a false ID
names = {
    0: 'Mohamed Atta',
    1: 'Nawaf Alhazmi',
    2: 'Hani Hanjour',
    3: 'Marwan Al-Shehhi',
    4: 'Ziad Jarrah',
    5: 'Mustafa al-Hisawi',
    6: 'Salem Alhazmi*',
    7: 'Lotfi Raissi',
    8: 'Saeed Alghamdi*',
    9: 'Abdul Aziz Al-Omari*',
    10: 'Hamza Alghamdi',
    11: 'Ramzi Bin al-Shibh',
    12: 'Said Bahaji',
    13: 'Ahmed Al Haznawi',
    14: 'Zakariya Essabar',
    15: 'Agus Budiman',
    16: 'Khalid Al-Mihdhar',
    17: 'Ahmed Alnami',
    18: 'Mounir El Motassadeq',
    19: 'Fayez Ahmed',
    20: 'Mamoun Darkazanli',
    21: 'Zacarias Moussaoui',
    22: 'Ahmed Khalil Ibrahim Samir Al-Ani',
    23: 'Abdussattar Shaikh',
    24: 'Osama Awadallah',
    25: 'Mohamed Abdi',
    26: 'Rayed Mohammed Abdullah',
    27: 'Bandar Alhazmi',
    28: 'Faisal Al Salmi',
    29: 'Mohand Alshehri*',
    30: 'Majed Moqed',
    31: 'Waleed Alshehri',
    32: 'Nabil al-Marabh',
    33: 'Raed Hijazi',
    34: 'Ahmed Alghamdi',
    35: 'Satam Suqami',
    36: 'Wail Alshehri'
}

The code below produces the adjacency matrix of the graph $G$ in Figure 2, a $37\times 37$ matrix $A$ with 

\begin{equation*}
A_{ij} = \begin{cases}
1 & \text{if vertices $i$ and $j$ are connected by an edge},\\
0 & \text{otherwise}.
\end{cases}
\end{equation*}

In [ ]:
# Initialize matrix of zeros
A = np.zeros([37,37])

# Change entries to reflect edges
A[0, [3, 9, 20, 14, 12, 11, 18, 21, 15, 22, 7, 4, 5, 2, 1]] = 1
A[1, [0, 6, 2, 16, 24, 23, 25, 17, 10, 8]] = 1
A[2, [1, 16, 30, 28, 27, 26, 7, 0, 4, 3]] = 1
A[3, [19, 6, 2, 4, 7, 0, 15, 11, 18, 12, 14, 20, 9, 5]] = 1
A[4, [13, 6, 2, 7, 15, 11, 0, 12, 14, 3]] = 1
A[5, [19, 3, 0, 31]] = 1
A[6, [1, 4, 3]] = 1
A[7, [2, 26, 0, 3, 4]] = 1
A[8, [32, 17, 10, 1, 13, 33]] = 1
A[9, [3, 0, 31]] = 1
A[10, [34, 17, 1, 13, 29, 8]] = 1
A[11, [3, 0, 4, 15, 21, 18, 12, 14]] = 1
A[12, [0, 11, 18, 20, 14, 3, 4]] = 1
A[13, [8, 10, 4]] = 1
A[14, [3, 4, 0, 11, 12]] = 1
A[15, [4, 11, 3, 0]] = 1
A[16, [23, 24, 2, 1]] = 1
A[17, [1, 10, 8]] = 1
A[18, [11, 12, 3, 0]] = 1
A[19, [29, 3, 5]] = 1
A[20, [3, 0, 12]] = 1
A[21, [11, 0]] = 1
A[22, [0]] = 1
A[23, [24, 16, 1]] = 1
A[24, [16, 1, 23]] = 1
A[25, [1]] = 1
A[26, [28, 27, 7, 2]] = 1
A[27, [26, 2]] = 1
A[28, [26, 2]] = 1
A[29, [19, 10]] = 1
A[30, [2]] = 1
A[31, [35, 5, 9, 36]] = 1
A[32, [34, 8, 33, 35]] = 1
A[33, [32, 8, 35]] = 1
A[34, [10, 32]] = 1
A[35, [32, 33, 31, 36]] = 1
A[36, [31, 35]] = 1

The Python package NetworkX contains many tools to deal with graphs. After defining the adjacency matrix $A$, the following code defines the graph $G$ and obtains lists of the nodes and edges of the graph. 

In [ ]:
# Import networkx
import networkx as nx

# Define graph G using adjacency matrix A
G = nx.Graph(A)

# Print nodes and edges
print(f'Nodes: {G.nodes()}')
print(f'Edges: {G.edges()}')

### Degree centrality

The _degree_ of a vertex is the number of edges incident to (i.e., that contain) the vertex. Since there are $N = 37$ nodes in the graph $G$ and no edges go from a vertex to the same vertex, each vertex can be connected to at most $N-1=36$ other vertices. It is common to divide the degree by the maximum possible degree to get a normalized degree value between 0 and 1 for each vertex; this helps mainly when comparing vertices in different graphs. This normalized degree value is called _degree centrality_.

Ranking each conspirator by their degree centrality is the simplest way to determine influence. Those with the highest degree centrality have the most contacts and are potentially the most influential members of the network. The NetworkX method `degree_centrality(G)` returns a dictionary associating the degree centrality to each vertex in $G$. 

### Closeness centrality

Closeness is another measure of influence in a graph $G$. Two vertices are distance $k$ apart if there is a path of $k$ edges linking the two vertices and no smaller path linking them. Two vertices that are not linked by a path are defined to be distance $\infty$ away from each other. Denote the distance between vertices $u$ and $v$ by $d(u,v)$. The _closeness_ $c(v)$ of a vertex $v$ is defined to be the reciprocal of the sum of the distances from $v$ to each of the vertices: 

\begin{equation*}
c(v) = \left[ \sum_{u} d(u,v)\right]^{-1}. 
\end{equation*}

It is common to replace the sum of the distances by the average distance. This amounts to multiplying the closeness by one less than the number of vertices to produce a number between 0 and 1. The _closeness centrality_ of a  vertex $v$ is

\begin{equation*}
C(v) = (N-1)c(v).
\end{equation*}

The closeness centrality can be used to determine influence in the graph; those conspirators with higher closeness centrality  are more closely related to the other conspirators and are able to spread information through the network quickly.  The NetworkX method `closeness_centrality(G)` returns a dictionary associating the closeness centrality to each vertex in $G$. 

### Betweenness centrality

A third measure of influence, betweenness, was introduced by the sociologist Linton Freeman in 1977. The betweenness of a vertex $v$ is related to the number of shortest paths between all other pairs of vertices that pass through the vertex $v$. Specifically, the _betweenness_ $b(v)$ of a vertex $v$ is computed as 

\begin{equation*}
b(v) = \sum \frac{\text{number of shortest paths between $s$ and $t$ that pass through $v$}}{\text{number of shortest paths between $s$ and $t$}},
\end{equation*}

where the sum is taken over all unordered pairs $\{s,t\}$ that don't include $v$. It is common to divide the betweenness $b(v)$ by the number of unordered pairs $\binom{N-1}{2} = (N-1)(N-2)/2$ to produce a number between 0 and 1 to obtain the _betweenness centrality_ $B(v)$. Conspirators that have high betweenness centrality are influential in the sense that a high proportion of the closest relationships in the network are mediated through them. The NetworkX method `betweenness_centrality(G)` returns a dictionary associating the betweenness centrality to each vertex in $G$.  

### PageRank

A fourth measure of influence was developed by the founders of Google. Their algorithm, PageRank, ranks webpages based on the link structure of the web, but it applies more generally to any digraph. Like a graph, a digraph (directed graph) has vertices but now each edge also carries a direction, so each edge has a start vertex and an end vertex. Directed edges are often called arcs, and are usually represented by arrows. Given a graph, we can make it into a digraph by replacing each edge with two directed edges, going in opposite directions, so PageRank can be applied to graphs as well. 

To describe PageRank, imagine that each vertex is a website and the directed edges from a given vertex point to websites to which our start vertex links. A user randomly clicks on links moving about the web, as follows. If they are at a webpage, then with probability $d$ they click on a link from that webpage. Each link from our webpage is equally likely to be chosen. With probability $1-d$ the user selects any of the webpages in the network; again, each website is equally likely to be chosen. 

We've been describing how an individual user behaves but we'd really like to apply these rules to many users distributed across the network. Imagine that we start with a population distribution  $R^0 \in [0,1]^N$ (Note: This requires that the sum of the entries in $R^0$ is 1) on a network with $N$ webpages so that $R^0_i$ equals the proportion of users initially on webpage $i$ (the exponent 0 indicates that we are at the starting point for an evolving system $R^t$). Now increase $t$ by 1 and allow each user to move to a new webpage, as described above.
The expected new population distribution is given by $R^{t+1} =  M R^t,$ 
where $M$ is a matrix with 

\begin{equation*}
M_{ij} = \left\{ \begin{array}{ll} \frac{d}{L_j} + \frac{1-d}{N}  & \text{if page $j$ links to page $i$} \\ \frac{1-d}{N} & \text{otherwise} \end{array} \right.
\end{equation*}

and $L_j$ is the number of outgoing links on page $j$. The matrix $M$ is called the modified adjacency matrix of $G$ with damping factor $d$. It will be convenient later to know an equivalent way to compute this matrix: 

\begin{equation*}
M = d(K^{-1}A)^T + \frac{1-d}{N}E,
\end{equation*}

where $K$ is the matrix with the outdegrees (number of edges emerging from a vertex) along the diagonal and $E$ is the $N\times N$ matrix filled with 1's. It turns out that if $d>0$, then this process converges, to a particular vector $R$, independent of the initial distribution of users on the network. This is a consequence of the Perron-Frobenius Theorem on eigenvalues of (stochastic) matrices. We won't go into the details of that theorem. However, note that since the steady state vector $R$ must satisfy the equation $R = MR$, we see immediately that $R$ is an eigenvector of $M$ corresponding to eigenvalue 1. The Perron-Frobenius Theorem implies that there is a unique eigenvector of $M$ with eigenvalue 1 with 1-norm equal to 1 whose entries are all positive. This is good since the $i^\text{th}$ entry $R_i$ of the unit eigenvector $R$ should be interpreted as the proportion of users that end up on webpage $i$ in the long run. The entries of this eigenvector rank the importance of each of the vertices in the network; higher entries correspond to more important vertices. (Note: This eigenvector has been called the 25 billion dollar eigenvector, the value of Google when it went public in 2004. For more details on the Pagerank algorithm, see [Bryant and Leise's paper](https://www.rose-hulman.edu/~bryan/googleFinalVersionFixed.pdf).

The NetworkX method `pagerank(G, d)` returns a dictionary associating the PageRank value to each vertex in $G$.  

Finding an eigenvector of a large matrix can be pretty difficult. So, how does NetworkX compute the PageRank? The Perron-Frobenius Theorem ensures that the matrix $M$ has a simple maximum eigenvalue (the number 1 appears as a root of the characteristic polynomial with multiplicity 1) and all other eigenvalues have 2-norm strictly less than 1. If we decompose a random vector $v$ into its projection onto the eigenspaces then $M$ acts by leaving the component in the 1-eigenspace alone but shrinks each other component (multiplies those components by $\lambda$ with $\| \lambda \|_2 < 1$). It follows that $M^tv$ converges to a vector whose normalization is $R$. In practice, this is the method used to compute the principal eigenvector $R$; we keep multiplying by $M$ (and normalizing, dividing by $\| v \|_1$ so that the vector remains a probability distribution) until the 2-norm of the difference between successive approximations is less than a given tolerance $\epsilon$. Though this does not guarantee that the vector $v$ is within $\epsilon$ of the limiting value $R$, in practice $v$ will tend to be close to $R$. You'll implement this method in the assignment to check that the NetworkX method `pagerank(G,d)` gives a reasonable value. 

There are many more measures of centrality and influence in graphs, but the degree centrality, closeness centrality, betweenness centrality, and PageRank measures will work for our purposes. Your goal in the assignment will be to produce four ranked lists of the most influential conspirators using these four measures. 

---

## Your Assignment

### Part 0

One of the learning goals of this project is to have you gain some experience with navigating documentation. You may need to use parts of NetworkX and NumPy that we did not explicitly cover in class. You should refer to the [NetworkX documentation](https://networkx.github.io/documentation/stable/) and the [NumPy documentation](https://numpy.org/devdocs/) throughout this assignment.

### Part 1

Using the code given above that defined the adjacency matrix `A` and the graph $G$, print a statement to the screen counting the number of _undirected_ edges in `A`; that is, the edges $(s,t)$ and $(t,s)$ are considered the same and should only be counted once.

_Hint._ Read the NetworkX documentation to find a method that might be helpful here.

### Part 2

Compute the degree centrality measures for the graph. Print the names and the degree centrality (just report 3 decimal places) of the five most influential terrorists to the screen. 

_Hint._ You might consider using the code

```
idx = np.array(measure_list).argsort()[::-1]
``` 

which sets `idx` to an array of indices so that `measure_list[idx[0]]` returns the largest element of `measure_list`, `measure_list[idx[1]]` returns the next largest element of `measure_list`, etc.

### Part 3

Compute the closeness centrality measures for the graph. Print the names and the closeness centrality of the five most influential terrorists to the screen. 

### Part 4

Computing closeness centrality requires us to compute the distance between every pair of vertices. Compute the (shortest) distance between the vertex for Khalid Al-Mihdhar and the vertex for Zacarias Moussaoui. Print this distance and a shortest path (using names rather than vertex numbers) to the screen. Is there more than one such path? If so, print another to the screen. If not, print a statement to the screen that there is a unique path.

## Part 5

Compute the betweenness centrality measures for the graph. Print the names and the betweenness centrality of the five most influential terrorists to the screen. 

## Part 6

Compute the PageRank measures for the graph (use the default value $d = 0.85$). Print the names and the PageRank of the five most influential terrorists to the screen. 

## Part 7

_The note in Part 0 especially applies to this part!_

Create the modified adjacency matrix $M$ for the graph (use the default value $d = 0.85$). Make a random vector $v$ in $[0,1]^{37}$, normalize $v$ with its 1-norm, and print it to the screen. 

Iteratively set $v = M v$ and normalize $v$ with its 1-norm. Stop when the 2-norm of the difference between successive $v$ vectors is less than $\epsilon = 1 \times 10^{-6}$. Print the number of iterations required and the final vector $v$ to the screen. 

Compute the 2-norm of the difference between the vector $v$ and the vector output by PageRank is less than $\epsilon$. You should find this difference to be small. Print the difference to the screen. 

Be careful: we're using both 2-norms and 1-norms here. You need to use the correct norm in the correct places.

---

## When you're finished

- Make sure your notebook runs from top to bottom with no errors. One way to accomplish this is to click on __Kernel &#8594; Restart & Run All__. This will restart Python, and run your notebook from top to bottom.

- When you're ready, submit this notebook using the SM286D Submission Form linked on the [class website](https://www.usna.edu/users/math/uhan/sm286d/).

---

## Grading

Your work will be graded as follows (60 total points):

- Part 1 (2 points)
    - Print the correct number to the screen in a statement
- Part 2 (7 points)
    - Compute the degree centrality values (3), sort the list (2) and print to the screen (2)
- Part 3 (7 points)
    - Compute the closeness centrality values (3), sort the list (2) and print to the screen (2)
- Part 4 (5 points)
    - Print the correct distance and a path to the screen (2). Determine if the path is unique and print an appropriate statement to the screen (3).
- Part 5 (7 points)
    - Compute the betweenness centrality values (3), sort the list (2) and print to the screen (2) 
- Part 6 (7 points)
    - Compute the PageRank values (3), sort the list (2) and print to the screen (2) 
- Part 7 (20 points)
    - Make the matrix M (5), make random vector v (2), normalize v (1), iteratively multiply by M and normalize (4), stop at appropriate time (2), print values to screen (4). Verify your answer is close to the answer from Q6 (2). 
- All parts: Helpful comments throughout your code (5 points)